In [30]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import pandas as pd

In [2]:
column_names = ["Date", "Market", "Keyword", "Average Position", "CPC", "Clicks", "CTR",
                "Impressions","Cost"]

In [6]:
raw_dataset = pd.read_csv("toy_dataset.csv", names=column_names,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True)


/home/mohi/anaconda3/envs/tf-2.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3,4,5,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
raw_df = raw_dataset.drop([0])


In [12]:
raw_df.head()

,Date,Market,Keyword,Average Position,CPC,Clicks,CTR,Impressions,Cost
1,20120524,US-Market,secure online back up,0,0,0,0.00%,0,0
2,20120524,US-Market,agile management software,1,1.2,21.22,8.20%,260,25.45
3,20120524,US-Market,crm for financial,0,0,0,0.00%,0,0
4,20120524,US-Market,disaster recovery planning for it,0,0,0,0.00%,0,0
5,20120524,US-Market,tracking a vehicle,0,0,0,0.00%,0,0


In [14]:
df = raw_df.drop(["Date", "Average Position", "CTR", "Impressions", "Cost"], axis=1)

In [17]:
dataset = df.dropna()

In [45]:
dataset.head()

,Market,Keyword,CPC,Clicks
1,US-Market,secure online back up,0,0
2,US-Market,agile management software,1.2,21.22
3,US-Market,crm for financial,0,0
4,US-Market,disaster recovery planning for it,0,0
5,US-Market,tracking a vehicle,0,0


In [19]:
keyword = dataset["Keyword"]

In [23]:
tokenizer = tfds.features.text.Tokenizer()

In [25]:
vocabulary_set = set()


In [32]:
for v in keyword:
    some_tokens = tokenizer.tokenize(v)
    vocabulary_set.update(some_tokens)


In [33]:
vocab_size = len(vocabulary_set)

In [34]:
vocabulary_set

{'0',
 '100',
 '1031',
 '11',
 '125',
 '2008',
 '2010',
 '21',
 '3',
 '32',
 '360',
 '3g',
 '4',
 '49ers',
 '4g',
 '5530',
 '5d',
 '8560',
 'ATT',
 'Adsense',
 'American',
 'Angeles',
 'Asbestos',
 'Baines',
 'California',
 'Canada',
 'Cancer',
 'Chicago',
 'Credit',
 'Degrees',
 'Equifax',
 'Ernst',
 'Florida',
 'Google',
 'Illinois',
 'India',
 'Lexington',
 'Los',
 'MCSA',
 'Manhattan',
 'Maryland',
 'Miami',
 'Ohio',
 'Online',
 'Orlando',
 'Point',
 'SEO',
 'UK',
 'University',
 'Virginia',
 'Washington',
 'Wisconsin',
 'a',
 'accept',
 'access',
 'accident',
 'account',
 'accounts',
 'acid',
 'action',
 'activities',
 'ad',
 'administration',
 'adsense',
 'advance',
 'adverse',
 'advertising',
 'adword',
 'adwords',
 'aetna',
 'affiliate',
 'affordable',
 'agency',
 'agile',
 'airlines',
 'all',
 'american',
 'ameriquest',
 'and',
 'android',
 'angeles',
 'anhembi',
 'annual',
 'anti',
 'antonio',
 'apartments',
 'app',
 'appendicitis',
 'applebees',
 'appliance',
 'application',

In [35]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)


In [43]:
some_example = keyword.iloc[0]

In [44]:
some_example

'secure online back up'

In [42]:
encoded_example = encoder.encode(some_example)
print(encoded_example)

[633, 403, 651, 539]


In [48]:
def encode(text):
    encoded_text = encoder.encode(text)
    return encoded_text



In [51]:
encoded_keyword = keyword.map(encode)

In [57]:
encoded_keyword

1             [633, 403, 651, 539]
2                   [668, 81, 284]
3                   [345, 590, 21]
4         [656, 69, 272, 590, 690]
5                  [240, 359, 368]
                    ...           
356050               [1, 306, 607]
356051                  [847, 469]
356052        [839, 367, 149, 747]
356053              [63, 847, 782]
356054             [119, 362, 187]
Name: Keyword, Length: 352540, dtype: object

In [60]:
"to substitute the column in the previous df"
dataset["Keyword"] = encoded_keyword

/home/mohi/anaconda3/envs/tf-2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [70]:
dataset

,Market,Keyword,CPC
1,1,"[633, 403, 651, 539]",0
2,1,"[668, 81, 284]",1.2
3,1,"[345, 590, 21]",0
4,1,"[656, 69, 272, 590, 690]",0
5,1,"[240, 359, 368]",0
...,...,...,...
356050,0,"[1, 306, 607]",0
356051,0,"[847, 469]",0
356052,0,"[839, 367, 149, 747]",0
356053,0,"[63, 847, 782]",0


In [66]:
target = dataset.pop("Clicks")

In [73]:
target.astype(np.float32)

1          0.000000
2         21.219999
3          0.000000
4          0.000000
5          0.000000
            ...    
356050     0.000000
356051     0.000000
356052     0.000000
356053     0.000000
356054     0.000000
Name: Clicks, Length: 352540, dtype: float32

In [68]:
dataset['Market'] = pd.Categorical(dataset['Market'])
dataset['Market'] = dataset.Market.cat.codes

/home/mohi/anaconda3/envs/tf-2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/mohi/anaconda3/envs/tf-2.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [77]:
dataset.values

array([[1, list([633, 403, 651, 539]), '0'],
       [1, list([668, 81, 284]), '1.2'],
       [1, list([345, 590, 21]), '0'],
       ...,
       [0, list([839, 367, 149, 747]), 0.0],
       [0, list([63, 847, 782]), 0.0],
       [0, list([119, 362, 187]), 0.0]], dtype=object)

In [74]:
dataset = tf.data.Dataset.from_tensor_slices((dataset.values.astype(np.float32), target.values.astype(np.float32)))

ValueError: setting an array element with a sequence.